In [1]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
import os, os.path
from tqdm import tqdm
import problem_unittests as tests
import tarfile
from plyfile import PlyData, PlyElement
import pandas as pd
import cv2

In [5]:
import tensorflow as tf

In [2]:
def conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides):
    height = conv_ksize[0]
    width = conv_ksize[1]
    F_W = tf.Variable(tf.truncated_normal([height, width, x_tensor.shape.as_list()[3], conv_num_outputs],stddev=0.01))# (height, width, input_depth, output_depth)
    F_b = tf.Variable(tf.zeros(conv_num_outputs))
    
    conLayer = tf.nn.conv2d(x_tensor, F_W, strides=[1,conv_strides[0], conv_strides[1],1], padding='SAME')
    conLayer = tf.nn.bias_add(conLayer, F_b)
    conLayer = tf.nn.relu(conLayer)
    return tf.nn.max_pool(conLayer, ksize=[1,pool_ksize[0],pool_ksize[1],1], strides=[1,pool_strides[0], pool_strides[1],1], padding='SAME')

In [3]:
def neural_net_image_input(image_shape):

    return tf.placeholder(tf.float32, shape=(None,image_shape[0],image_shape[1],image_shape[2]),name="x")


def neural_net_label_input(n_classes):

    return tf.placeholder(tf.float32,shape=(None,n_classes),name="y")


def neural_net_keep_prob_input():

    return tf.placeholder(tf.float32, shape=None, name="keep_prob")

In [7]:
def conv_net(x, keep_prob):

    pool1 = conv2d_maxpool(x, 64, [3,3], [1,1], [8,8], [1,1])
    pool2 = conv2d_maxpool(pool1, 28, [3,3], [1,1], [5,5], [1,1])
    pool2 = tf.nn.dropout(pool2, keep_prob)
    
    flat = tf.contrib.layers.flatten(pool2)

    layer3 = tf.contrib.layers.fully_connected(flat,1024)
    layer4 = tf.contrib.layers.fully_connected(layer3,1024)
    layer4 = tf.nn.dropout(layer4, keep_prob)
    
    #Apply an Output Layer
    out = tf.contrib.layers.fully_connected(layer4,32,activation_fn=None)
    
    return out


# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x = neural_net_image_input((64, 64, 3))
y = neural_net_label_input(10)
keep_prob = neural_net_keep_prob_input()

# Model
logits = conv_net(x, keep_prob)

# Name logits Tensor, so that is can be loaded from disk after training
logits = tf.identity(logits, name='logits')

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
def train_neural_net(session, optimizer, keep_probability, feature_batch, label_batch):

    session.run(optimizer, feed_dict={
        x : feature_batch,
        y : label_batch,
        keep_prob : keep_probability
    })